In [ ]:
import os
import meshio
import numpy as np
from scipy.spatial import cKDTree
from scipy.ndimage import gaussian_filter


np.seterr(divide='ignore', invalid='ignore')

## Get vtk mesh data

In [ ]:
paleofile = 'earth/bio_vtk/elev100Ma.vtk'
mesh = meshio.read(paleofile)

vertices = mesh.points
cells = mesh.cells_dict['triangle']
elev = mesh.point_data['Z']
cost = mesh.point_data['cost']

In [ ]:
# paleofile = 'earth/elev_npz/elev0Ma.npz'
# topo = np.load(paleofile)
# cells = topo['c']
# vertices = topo['v']
# elev = topo['z']
# cost = topo['cost']

lonlatfile = 'earth/elev_npz/icoord_elev.npz'
mesh_lonlat = np.load(lonlatfile)
rcoord = mesh_lonlat['v']
lons = np.radians(rcoord[0])
lats = np.radians(rcoord[1])

## Build lon/lat structured grid

Define the resolution in degree for regular grid resolution

In [ ]:
res = 0.1 

In [ ]:
def buildReg(res):
    
    # Regular lon/lat coordinates
    lon = np.arange(0.0, 360.0, res)
    lat = np.arange(0.0, 180.0, res)

    # Build the mesh
    mlon, mlat = np.meshgrid(lon, lat, sparse=False, indexing="ij")
    nbPts = mlon.size

    # Define regular mesh information
    points = np.zeros((nbPts, 2))
    points[:, 0] = np.radians(mlon.flatten())
    points[:, 1] = np.radians(mlat.flatten())
    
    return points, mlon.shape

regPts, rShape = buildReg(res)

## Perform interpolation to regular grid

Build unstructured mesh kd-tree for smoothing

In [ ]:
utree = cKDTree(list(zip(lons, lats)))
uDist, uIds = utree.query(regPts, k=3)

Map data from the unstructured grid on the structured one by performing inverse weighting distance...

In [ ]:
uWght = 1.0 / uDist ** 2
uIs = np.where(uDist[:, 0] == 0)[0]

regData = np.sum(uWght * cost[uIds][:, :], axis=1) / np.sum(uWght, axis=1)
if len(uIs) > 0:
    regData[uIs] = cost[uIds[uIs, 0]]

## Apply smoothing on unstructured grid

In [ ]:
smth = 4

In [ ]:
regData = gaussian_filter(regData.reshape(rShape), sigma=smth).flatten()

## Perform interpolation to unstructured grid

In [ ]:
rtree = cKDTree(regPts)
rDist, rIds = rtree.query(list(zip(lons, lats)), k=3)

In [ ]:
rWght = 1.0 / rDist ** 2
rIs = np.where(rDist[:, 0] == 0)[0]

sCost = np.sum(rWght * regData[rIds][:, :], axis=1) / np.sum(rWght, axis=1)
if len(rIs) > 0:
    sCost[rIs] = regData[rIds[rIs, 0]]

In [ ]:
sCost[elev<0] = 0.

In [ ]:
paleomesh = 'earth/bio_vtk/smooth_elev100Ma.vtk'
vtk_mesh = meshio.Mesh(vertices, {'triangle': cells}, 
                       point_data={"Z":elev, "cost":cost, "scost":sCost, "ncost":sCost/(sCost.max())})
meshio.write(paleomesh, vtk_mesh)

print("Writing VTK file {}".format(paleomesh))

In [ ]:
sCost.max()